# ZIPCODE - GDP conversion

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

In [3]:
accept_add = '../data/LC/Lending_Club_Accepted_2014_2018.csv'
reject_add = '../data/LC/Lending_Club_Rejected_2014_2018.csv'

/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_15562/1522798249.py:3: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(accept_add)


In [16]:
df1 = pd.read_csv(accept_add)
df2 = pd.read_csv(reject_add)

/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/2660902038.py:1: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(accept_add)


In [17]:
def to_year(x):
    return pd.to_datetime(x).year

df1['issue_y'] = df1['issue_d'].apply(to_year)
df1['issue_y'] = df1['issue_y'].apply(str)

In [19]:
import pandas as pd

#read df1

path = '../data/gdp/ZIP-COUNTY-FIPS_2017-06.csv' #path of zip file - county fips 

#read fips-zip file
df_fips = pd.read_csv(path, encoding = "ISO-8859-1", dtype = object)

In [20]:
#add new column of only issue year 
def to_year(x):
    return pd.to_datetime(x).year

df1['issue_y'] = df1['issue_d'].apply(to_year)
df1['issue_y'] = df1['issue_y'].apply(str)


KeyboardInterrupt



In [21]:
#change zip code to 345xx format
def f(x): 
    return x[:3] + 'xx'

df_fips['ZIP'] = df_fips['ZIP'].apply(f)

ziptofip = dict(zip(df_fips.ZIP, df_fips.STCOUNTYFP))  


In [22]:
#check if there are corredsponding fips in zip code
def g(x):
    if x in ziptofip.keys():
        return 1
    else:
        return 0

df1['haszip'] = df1['zip_code'].apply(g)

In [23]:
df1['haszip'].sum()

2029524

In [403]:
#zipcode where no match in our zip-fip dataset found
df1.loc[df1['haszip'] == 0]['zip_code'].unique()

array(['962xx', '348xx', '092xx', '819xx', '965xx', '742xx', '096xx',
       '097xx', '098xx', '963xx', '555xx', '094xx', '090xx', '091xx',
       '861xx', '269xx', '892xx', '340xx', '929xx', '702xx', '709xx',
       '849xx', '694xx', '854xx', '771xx', '817xx', '568xx', '343xx',
       '862xx', '942xx', '093xx', '858xx', '095xx', '987xx', '885xx',
       '663xx', '696xx', '621xx', '732xx', '345xx', '966xx', '901xx',
       '353xx', '872xx', '429xx', '682xx', '888xx', '889xx', '909xx',
       '964xx', '698xx', nan, '552xx', '896xx'], dtype=object)

In [24]:
#for all gdp files, generate below dictionary
gdp_dict = dict() #zip to [year2014gdp, year2015gdp, ..., year 2018gdp]
import os
# assign directory
directory = '../data/gdp/CAGDP1/' #your GDP file directory
fiptozip = dict()

# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if not filename.startswith('.') and os.path.isfile(f):
        df = pd.read_csv(f, encoding = "ISO-8859-1")
        
        def tostr(x):
            try:
                res = str(x[2:][:-1])
                assert(len(res) == 5)
            except:
                res = ""

            return res
        
        #print(df.columns)
        df['GeoFIPS'] = df['GeoFIPS'].apply(tostr)
        
        df = df.loc[df['Description'] =='Real GDP (thousands of chained 2012 dollars)']
        tmp = dict(zip(df.GeoFIPS.apply(str), df[ ['2014', '2015', '2016', '2017', '2018']].values))
        fiptozip.update(tmp)
        

In [35]:
#get a new dictonary that has only corresponded zipcode-fips
df_tmp = df1.loc[df1['haszip'] == 1]

def j(x):
    return ziptofip[x]
df_tmp['FIPS'] = df_tmp['zip_code'].apply(j)

/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/2444950112.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['FIPS'] = df_tmp['zip_code'].apply(j)


In [26]:
#get a new column of gdp
def k(x):
    fip = ziptofip[x]
    if fip in fiptozip:
        return tuple(fiptozip[fip])
    else:
        return -1
    
df_tmp['gdp'] = df_tmp['zip_code'].apply(k)

/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/3424655678.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['gdp'] = df_tmp['zip_code'].apply(k)


In [27]:
#get rid of nan data
df_tmp = df_tmp.loc[df_tmp['gdp'] != -1]

In [28]:
#get final gdp
def fxy(x, y):
    ind = int(x)-2014
    return y[ind]

df_tmp['gdp'] = df_tmp.apply(lambda x: fxy(x['issue_y'], x['gdp']), axis=1)

/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/2293908101.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['gdp'] = df_tmp.apply(lambda x: fxy(x['issue_y'], x['gdp']), axis=1)


In [29]:
#get a unemployment insurance dictionary dictionary
inc_dict = dict() #zip to [year2014gdp, year2015gdp, ..., year 2018gdp]
import os
# assign directory
directory = '../data/gdp/CAINC30/' #your GDP file directory
#fiptozip = dict()

# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if not filename.startswith('.') and os.path.isfile(f):
        df = pd.read_csv(f, encoding = "ISO-8859-1")
        
        def tostr(x):
            try:
                res = str(x[2:][:-1])
                assert(len(res) == 5)
            except:
                res = ""

            return res
        
        #print(df.columns)
        df['GeoFIPS'] = df['GeoFIPS'].apply(tostr)
        
        df = df.loc[df['Description'] =='  Unemployment insurance compensation']
        tmp = dict(zip(df.GeoFIPS.apply(str), df[ ['2014', '2015', '2016', '2017', '2018']].values))
        inc_dict.update(tmp)

/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/1075590900.py:14: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding = "ISO-8859-1")
/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/1075590900.py:14: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding = "ISO-8859-1")


In [30]:
#get unemployment dictionary
def k(x):
    fip = ziptofip[x]
    if fip in inc_dict:
        return tuple(inc_dict[fip])
    else:
        return -1
    
df_tmp['unemployment_insurance'] = df_tmp['zip_code'].apply(k)

In [31]:
#get dividend/interest/rent dictionary
inc_dict = dict() #zip to [year2014gdp, year2015gdp, ..., year 2018gdp]
import os
# assign directory
directory = '../data/gdp/CAINC30/' #your GDP file directory
#fiptozip = dict()

# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if not filename.startswith('.') and os.path.isfile(f):
        df = pd.read_csv(f, encoding = "ISO-8859-1")
        
        def tostr(x):
            try:
                res = str(x[2:][:-1])
                assert(len(res) == 5)
            except:
                res = ""

            return res
        
        #print(df.columns)
        df['GeoFIPS'] = df['GeoFIPS'].apply(tostr)
        
        df = df.loc[df['Description'] ==' Dividends, interest, and rent 2/']
        tmp = dict(zip(df.GeoFIPS.apply(str), df[ ['2014', '2015', '2016', '2017', '2018']].values))
        inc_dict.update(tmp)


/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/3464197326.py:14: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding = "ISO-8859-1")
/var/folders/ml/b32sv3j51m99nh7xvczpn81r0000gn/T/ipykernel_6418/3464197326.py:14: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding = "ISO-8859-1")


In [32]:
#get it in the df columns
def k(x):
    fip = ziptofip[x]
    if fip in inc_dict:
        return tuple(inc_dict[fip])
    else:
        return -1
    
df_tmp['div_rent_int'] = df_tmp['zip_code'].apply(k)

In [33]:
#final change
def fxy(x, y):
    ind = int(x)-2014
    return y[ind]

df_tmp['unemployment_insurance'] = df_tmp.apply(lambda x: fxy(x['issue_y'], x['unemployment_insurance']), axis=1)
df_tmp['div_rent_int'] = df_tmp.apply(lambda x: fxy(x['issue_y'], x['div_rent_int']), axis=1)

0          42101
1          46105
2          17197
3          34003
4          42133
           ...  
2029947    01103
2029948    49057
2029949    06059
2029950    12099
2029951    48201
Name: FIPS, Length: 2029524, dtype: object

In [ ]:
#convert to a dictionary